In [21]:
# import os
# os.chdir('./data')

import pandas as pd
import numpy as np
df = pd.read_csv('페이지내_사용자_이동.csv', engine='python')

In [22]:
df.head()
# ex) 0번 고객: 페이지 C->E->B->F->C 이런 순으로 방문한 것

,고객ID,방문 페이지,순서
0,0,페이지C,1
1,0,페이지E,2
2,0,페이지B,3
3,0,페이지F,4
4,0,페이지C,5


In [23]:
# 주문별 카트에 추가한 순서를 고려하기위해 정렬 필요
df.sort_values(by=['고객ID', '순서'], inplace=True)

In [24]:
page_set = df['방문 페이지'].unique()
page_set[:10]

array(['페이지C', '페이지E', '페이지B', '페이지F', '페이지D', '페이지J', '메인', '페이지G',
       '페이지A', '페이지I'], dtype=object)

In [25]:
page_sequence_per_order = df.groupby('고객ID')['방문 페이지'].apply(np.array)

In [26]:
page_sequence_per_order.head()

고객ID
0    [페이지C, 페이지E, 페이지B, 페이지F, 페이지C, 페이지D, 페이지J, 메인,...
1    [페이지B, 메인, 메인, 페이지A, 페이지F, 페이지C, 페이지I, 페이지E, 페...
2    [페이지F, 페이지J, 페이지D, 페이지G, 페이지C, 메인, 페이지I, 페이지J,...
3    [페이지I, 페이지I, 페이지J, 페이지J, 페이지C, 페이지A, 페이지H, 페이지...
4    [페이지J, 페이지H, 페이지G, 페이지G, 페이지E, 페이지C, 페이지A, 페이지...
Name: 방문 페이지, dtype: object

In [27]:
from itertools import product # 가능한 모든 조합을 출력해줌

def contain_pattern(record, pattern, L):
    # 특정 record하에 특정 pattern이 발생했는지
    # L:윈도우 크기
    output = False
    if set(record) & set(pattern) != set(pattern): # pattern에 포함된 모든 아이템 집합이 record에 포함된 아이템 집합에 속하지 않으면
        # 해석: record와 pattern의 교집합이 pattern과 같지 않다 => pattern이 record에 속하지 않는다
        return False
    
    else:
        # 패턴에 속한 개별 아이템에 대한 위치를 미리 구하기
        # pattern에 있는 item을 돌면서 record의 어느 위치에 있는지 확인
        # [0]: np.where의 output이 array임. 그중 0번째를 가져오겠다는 것
        pattern_index_list = [np.where(record == item)[0] for item in pattern]
        
        ## 가능한 모든 조합에서 위치 간 거리가 L이하면 True를 반환
        # Ex) record = [A, B, C, A, C, C], pattern = [A, B], L = 1
        # A의 위치: [0, 3], B의 위치: [1]
        # 가능한 모든 조합(product): [0, 1], [3, 1](얘는 A가 3번째 있고 B가 첫번째 있으므로 L에 상관없이 우리가 원하는 패턴이 아냐 =>그래서 밑에 distance를 구함)
        # 가능한 모든 조합의 거리 차이: [1-0, 1-3] 중에 0번째 요소는 만족하므로 True
        
        for pattern_index in product(*pattern_index_list):
            # product는 모든 조합을 찾음
            distance = np.array(pattern_index)[1:] - np.array(pattern_index)[:-1]
            # [1:]는 뒤에 있는 요소, [:-1]는 앞에 있는 요소
            if sum((distance <= L) & (distance > 0)) == (len(pattern_index) - 1): # -1은 맨 앞에 있는 것은 거리상 무시됐기때문에
                output = True
                break
                
        return output

In [28]:
def find_maximum_frequent_sequence_item(item_set, sequence_data, min_support=0.01, L=1):
    queue = []
    maximum_frequent_sequence_item = []
    
    # 유니크한 아이템 집합에 대해, min_support가 넘는 아이템들만 queue에 추가
    for item in item_set:
        occurence = sequence_data.apply(contain_pattern, pattern=[item], L=L).sum()
        if occurence / len(sequence_data) >= min_support:
            # 등장비율
            queue.append([item])
            
    while queue:
        current_pattern = queue.pop()  # 맨 마지막 값 빼기
        check_maximum_frequent = True # 모든 자식 집합이 min_support를 넘기지 않으면 True를 유지
        for item in item_set:
            occurence = sequence_data.apply(contain_pattern, pattern=current_pattern + [item], L=L).sum()
            if occurence / len(sequence_data) >= min_support: # min_support를 넘는 패턴을 queue에 추가
                check_maximum_frequent = False
                queue.append(current_pattern + [item])
                
        if check_maximum_frequent and len(current_pattern) > 1:
            maximum_frequent_sequence_item.append(current_pattern)
            
    return maximum_frequent_sequence_item

In [29]:
def generate_association_rules(maximum_frequent_sequence_item, sequence_data, min_support = 0.01, min_confidence = 0.5, L = 1):
    # 결과 초기화
    result = {"부모":[], "자식":[], "지지도":[], "신뢰도":[]}
        
    for sequence_item in maximum_frequent_sequence_item:
        # A -> B에서 A, B를 모두 포함하는 가짓 수 co_occurence 계산
        co_occurence = sequence_data.apply(contain_pattern, pattern = sequence_item, L = L).sum()
        support = co_occurence / len(sequence_data)
        if co_occurence > min_support:
            for i in range(len(sequence_item)-1, 0, -1): # 한 아이템 집합에 대해, 부모의 크기를 1씩 줄여나가는 방식으로 부모와 자식 설정
                antecedent = sequence_item[:i]
                consequent = sequence_item[i:]
                antecedent_occurence = sequence_data.apply(contain_pattern, pattern = antecedent, L = L).sum()
                
                confidence = co_occurence / antecedent_occurence
                if confidence > min_confidence:
                    result['부모'].append(antecedent)
                    result['자식'].append(consequent)
                    result['지지도'].append(support)
                    result['신뢰도'].append(confidence)
            
    return pd.DataFrame(result)

In [32]:
page_set

array(['페이지C', '페이지E', '페이지B', '페이지F', '페이지D', '페이지J', '메인', '페이지G',
       '페이지A', '페이지I', '페이지H'], dtype=object)

In [33]:
page_sequence_per_order

고객ID
0     [페이지C, 페이지E, 페이지B, 페이지F, 페이지C, 페이지D, 페이지J, 메인,...
1     [페이지B, 메인, 메인, 페이지A, 페이지F, 페이지C, 페이지I, 페이지E, 페...
2     [페이지F, 페이지J, 페이지D, 페이지G, 페이지C, 메인, 페이지I, 페이지J,...
3     [페이지I, 페이지I, 페이지J, 페이지J, 페이지C, 페이지A, 페이지H, 페이지...
4     [페이지J, 페이지H, 페이지G, 페이지G, 페이지E, 페이지C, 페이지A, 페이지...
                            ...                        
95    [페이지D, 페이지H, 페이지I, 메인, 페이지B, 페이지E, 페이지F, 페이지G,...
96    [페이지E, 페이지I, 페이지E, 페이지G, 페이지A, 페이지E, 페이지D, 페이지...
97    [페이지B, 페이지B, 메인, 메인, 페이지B, 페이지F, 페이지J, 페이지D, 페...
98                               [메인, 페이지D, 페이지F, 페이지G]
99    [페이지E, 페이지H, 페이지B, 페이지D, 페이지B, 페이지A, 페이지I, 페이지...
Name: 방문 페이지, Length: 100, dtype: object

In [30]:
maximum_frequent_sequence_item = find_maximum_frequent_sequence_item(page_set, page_sequence_per_order, min_support = 0.02, L = 1)

In [34]:
maximum_frequent_sequence_item

[['페이지H', '페이지H'],
 ['페이지H', '페이지I'],
 ['페이지H', '페이지A', '페이지D'],
 ['페이지H', '메인'],
 ['페이지H', '페이지J'],
 ['페이지H', '페이지D', '페이지F'],
 ['페이지H', '페이지D', '페이지E'],
 ['페이지H', '페이지F', '페이지E'],
 ['페이지H', '페이지B', '페이지D'],
 ['페이지H', '페이지E'],
 ['페이지H', '페이지C', '페이지I'],
 ['페이지H', '페이지C', '페이지J'],
 ['페이지I', '페이지H', '페이지E'],
 ['페이지I', '페이지I'],
 ['페이지I', '페이지A'],
 ['페이지I', '페이지G'],
 ['페이지I', '메인', '페이지B'],
 ['페이지I', '페이지J'],
 ['페이지I', '페이지D', '페이지F'],
 ['페이지I', '페이지F'],
 ['페이지I', '페이지B', '페이지C'],
 ['페이지I', '페이지E', '페이지E'],
 ['페이지I', '페이지C'],
 ['페이지A', '페이지H', '페이지B'],
 ['페이지A', '페이지I'],
 ['페이지A', '페이지A', '페이지H'],
 ['페이지A', '페이지G', '페이지J', '페이지E'],
 ['페이지A', '페이지G', '페이지B'],
 ['페이지A', '메인'],
 ['페이지A', '페이지J', '메인'],
 ['페이지A', '페이지J', '페이지J'],
 ['페이지A', '페이지D', '페이지B'],
 ['페이지A', '페이지F', '페이지C'],
 ['페이지A', '페이지B', '페이지F'],
 ['페이지A', '페이지B', '페이지E'],
 ['페이지A', '페이지E'],
 ['페이지A', '페이지C'],
 ['페이지G', '페이지A', '페이지J'],
 ['페이지G', '페이지A', '페이지D'],
 ['페이지G', '페이지A', '페이지B'],
 ['페이지G', '페이지G'],
 ['페이지G', '페이지J', '페이

In [31]:
result = generate_association_rules(maximum_frequent_sequence_item, page_sequence_per_order, min_support = 0.01, min_confidence = 0, L = 1)
result.sort_values(by = ['지지도', '신뢰도'], ascending = False)

,부모,자식,지지도,신뢰도
160,[페이지B],[페이지J],0.07,0.155556
59,[페이지A],[페이지C],0.07,0.148936
109,[페이지J],[페이지D],0.07,0.142857
115,[페이지J],[페이지C],0.07,0.142857
75,[메인],[페이지H],0.06,0.150000
...,...,...,...,...
205,[페이지C],"[메인, 페이지A, 페이지J]",0.02,0.037037
207,[페이지C],"[메인, 페이지E]",0.02,0.037037
209,[페이지C],"[페이지J, 페이지I]",0.02,0.037037
214,[페이지C],"[페이지E, 메인]",0.02,0.037037
